# 1. Response from AI tutor

In [69]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [70]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [63]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [75]:
system_prompt = """You are an Ai engineer, phd in mathematics, deep learning engineer, and expert in other topics"""
system_prompt += """
When responding, always use Markdown for formatting. For any code, use well-structured code blocks with syntax highlighting,
For instance:
```python

sample_list = [for i in range(10)]
```

Break down explanations into clear, numbered steps for better understanding. 
Highlight important terms using inline code formatting (e.g., `function_name`, `variable`).
Provide examples for any concepts and ensure all examples are concise, clear, and relevant.
Your goal is to create visually appealing, easy-to-read, and informative responses.

"""

In [76]:
def tutor_user_prompt(question):
    # Ensure the question is properly appended to the user prompt.
    user_prompt = (
        "Please carefully explain the following question in a step-by-step manner:\n\n"
    )
    user_prompt += question
    return user_prompt

In [77]:
def Tutor(question, MODEL):
    # Generate the user prompt dynamically.
    user_prompt = tutor_user_prompt(question)
    
    # OpenAI API call to generate response.
    if MODEL == 'gpt-4o-mini':
        print(f'You are getting response from {MODEL}')
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )
    else:
        MODEL == 'llama3.2'
        print(f'You are getting response from {MODEL}')
        stream = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )

    # Initialize variables for response processing.
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    # Process the response stream and update display dynamically.
    for chunk in stream:
        # Safely access the content attribute.
        response_chunk = getattr(chunk.choices[0].delta, "content", "")
        if response_chunk:  # Check if response_chunk is not None or empty
            response += response_chunk
            # No replacement of Markdown formatting here!
            update_display(Markdown(response), display_id=display_handle.display_id)

In [78]:
question = """
Please answer the question and explain the answer:
what is the capital of United states of america
"""

In [79]:
Tutor(question=question, MODEL=MODEL_GPT)

You are getting response from gpt-4o-mini


Sure! Let's break down the question step-by-step.

### Question:
**What is the capital of the United States of America?**

### Steps to Understand the Answer:

1. **Identify the Subject Matter**:
   - The question asks about the **capital city** of a specific country, in this case, the **United States of America (USA)**.

2. **Understanding "Capital"**:
   - A **capital** is defined as the city where the central government is located. It typically hosts government buildings, including the legislative, executive, and often the judicial branches.

3. **Historical Context**:
   - The capital of the USA was established in the late 18th century after the American Revolution. The site was chosen to appease both northern and southern states.

4. **Current Capital**:
   - The current capital of the United States of America is **Washington, D.C.**.

### Conclusion:
To directly answer the question: 

- The capital of the United States of America is **Washington, D.C.**

This answer not only provides the information requested but also contextualizes its significance. 

### Additional Information:
- **Washington, D.C.** stands for the District of Columbia and is a federal district, meaning it is not part of any state.
- It was named after George Washington, the first President of the United States.

Let me know if you have further questions or need more information!

# Double LLM Collaboration Approach

In [80]:
def refine_with_ollama(raw_question):
    """
    Use Ollama to refine the user's raw question into a well-structured prompt.
    """
    print("Refining the query using Ollama...")
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Refine and structure the following user input."},

        {"role": "user", "content": raw_question},
    ]
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages,
        stream=False  # Non-streamed refinement
    )
    refined_query = response.choices[0].message.content
    return refined_query

In [81]:
def ask_with_ollama_and_gpt(raw_question):
    """
    Use Ollama to refine the user query and GPT to generate the final response.
    """
    # Step 1: Refine the query using Ollama
    refined_query = refine_with_ollama(raw_question)
    
    # Step 2: Generate final response with GPT
    print("Generating the final response using GPT...")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": refined_query},
    ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True  # Stream response for dynamic display
    )

    # Step 3: Combine responses
    response = ""
    display_handle = display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:"), display_id=True)
    for chunk in stream:
        response_chunk = getattr(chunk.choices[0].delta, "content", "")
        if response_chunk:
            response += response_chunk
            update_display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:\n\n{response}"), display_id=display_handle.display_id)

In [84]:
# Example Usage
question = """
Please answer the question and explain it beautifully:
who is the founder of Technical Universtiy of Darmstadt
"""

In [83]:
ask_with_ollama_and_gpt(raw_question=question)

Refining the query using Ollama...
Generating the final response using GPT...


### Refined Query:

A straightforward question!

The founder of the Technical University of Darmstadt (Technische Universität Darmstadt, or TU Darmstadt) is Bernhard Heine.

However, I can provide more context and information about Bernhard Heine. He was a German engineer and politician who played a significant role in the founding of the Technische Hochschule Darmstadt, which later became the modern-day Technische Universität Darmstadt. Founded in 1877, it is one of the oldest technical universities in Germany.

Bernhard Heine was born on May 5, 1819, and died on March 12, 1900. Throughout his life, he worked tirelessly to establish a technical university that would provide students with advanced education in fields such as engineering, physics, and mathematics. Under his leadership, the Technische Hochschule Darmstadt grew to become one of Germany's premier technical universities.

Today, the Technische Universität Darmstadt is recognized for its research excellence and academic programs in a wide range of disciplines, including engineering, computer science, natural sciences, and humanities.

---

### Final Response:

Certainly! Here’s a concise summary of Bernhard Heine and the Technische Universität Darmstadt, broken down into clear steps:

### 1. **Founding Background**
   - **Founder:** Bernhard Heine
   - **Founded:** 1877
   - **Original Name:** Technische Hochschule Darmstadt

### 2. **Bernhard Heine**
   - **Birth Date:** May 5, 1819
   - **Death Date:** March 12, 1900
   - **Profession:** German engineer and politician
   - **Significance:** Instrumental in establishing a technical university focused on advanced education.

### 3. **University Evolution**
   - **Growth:** Under Heine's leadership, the university evolved into a major institution for engineering, physics, and mathematics.
   - **Modern Recognition:** It is now known as Technische Universität Darmstadt and is recognized for its academic excellence.

### 4. **Academic Focus**
   - **Disciplines Offered:**
     - Engineering
     - Computer Science
     - Natural Sciences
     - Humanities

### 5. **Current Standing**
   - **Reputation:** TU Darmstadt is esteemed for its research and educational programs, continuing the legacy initiated by Bernhard Heine.

### Example of Information Representation
Here’s a summarized example using a bullet point format:

- **Founder:** Bernhard Heine
- **Establishment Year:** 1877
- **Current Name:** Technische Universität Darmstadt
- **Key Fields:** Engineering, Computer Science, Natural Sciences, Humanities

### Additional Note
The impact of Bernhard Heine endures, as TU Darmstadt remains a pivotal institution in Germany's educational landscape, inspiring generations of students and researchers alike.